In [19]:
import pandas as pd

df = pd.read_csv("./Real_Time_Passenger_Information_Sign_Locations.csv")

df.keys()

Index(['BoroName', 'BoroCode', 'BoroCD', 'CounDist', 'AssemDist', 'StSenDist',
       'CongDist', 'StopName', 'Corner', 'BusStopID', 'Routes', 'Direction',
       'Notes', 'Elected Official', 'WO #', 'Wireless Communication',
       'Release Date', 'RTPI Complete', 'Latitude', 'Longitude', 'NTAName',
       'FEMAFldz', 'FEMAFldT', 'HrcEvac', 'Location'],
      dtype='object')

In [20]:
df.index

RangeIndex(start=0, stop=513, step=1)

In [21]:
df.drop_duplicates(subset=['Location'], inplace=True)

In [42]:
to_export_df = df.filter(items=['NTAName', 'Latitude', 'Longitude'])

In [44]:
to_export_df.to_csv('station_location.csv', index=False)

In [46]:
import csv
from shapely.wkt import loads

def parse_csv_file(file_path):
    # Initialize empty lists to store the extracted values
    latitudes = []
    longitudes = []
    names = []

    with open(file_path, 'r') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Skip the header row if present

        # Loop over each row in the csv using reader object
        for row in reader:
            # row variable is a list that represents a row in csv
            latitude = float(row[1])
            longitude = float(row[2])
            name = row[0]
            # Append the values to the respective lists
            latitudes.append(latitude)
            longitudes.append(longitude)
            names.append(name)

    return latitudes, longitudes, names

# Path to the CSV file
file_path = './bus_station_location.csv'

# Call the function to parse the data from the file
latitudes, longitudes, names = parse_csv_file(file_path)

coordinates = {}

for lat, lon, n in zip(latitudes, longitudes, names):
  coordinates[n] = [lat,lon]

print(coordinates)

{'Central Park': [40.769033, -73.969649], 'Sunset Park (Central)': [40.636607, -74.01154], 'Prospect Park': [40.66007, -73.977772], 'Upper East Side-Lenox Hill-Roosevelt Island': [40.766741, -73.956738], 'Gravesend (West)': [40.600434, -73.995232], 'Chelsea-Hudson Yards': [40.747927, -74.003847], 'Fort Wadsworth': [40.599129, -74.062971], 'East Elmhurst': [40.767857, -73.880212], 'Midtown South-Flatiron-Union Square': [40.74986, -73.987669], 'Sunnyside': [40.744165, -73.925245], 'Grasmere-Arrochar-South Beach-Dongan Hills': [40.598413, -74.0919], 'East Harlem (South)': [40.785357, -73.949465], 'University Heights (South)-Morris Heights': [40.850268, -73.915494], 'Midtown-Times Square': [40.75357, -73.98098], 'Carroll Gardens-Cobble Hill-Gowanus-Red Hook': [40.674894, -74.001108], 'West New Brighton-Silver Lake-Grymes Hill': [40.634584, -74.117749], 'Upper East Side-Yorkville': [40.774585, -73.950985], 'Cypress Hills': [40.681721, -73.871923], 'New Springville-Willowbrook-Bulls Head-Tra

In [51]:
import requests
import csv

# Create and write to the CSV file
with open("./station_distance.csv", mode="w", newline="") as file:
      writer = csv.writer(file)
      writer.writerow(["start", "end", "duration (mins)"])


with open("./station_distance.csv", mode="a", newline="") as file:
    writer = csv.writer(file)

    for start in coordinates.items():
      start_lat = start[1][0]
      start_lon = start[1][1]
      start_name = start[0]
      print(start_name)
      for end in coordinates.items():

        end_name = end[0]
        if end_name == start_name:
          continue

        end_lat = end[1][0]
        end_lon = end[1][1]

        url = f"https://api.mapbox.com/directions/v5/mapbox/driving/{start_lon}%2C{start_lat}%3B{end_lon}%2C{end_lat}?alternatives=false&annotations=distance%2Cduration&continue_straight=true&geometries=geojson&overview=full&steps=false&access_token=pk.eyJ1Ijoic2FzaGEtc3NtOTk2MCIsImEiOiJjbG4wcm9saG8xc3duMmlxd3pjY2MybDcyIn0.ol8PlZNYBX_XbtBu8I3igw"

        try:
            r = requests.get(url)
            r.raise_for_status()  # Raise an exception for bad response status
            dur = r.json()['routes'][0]['duration'] / 60  # Duration in minutes

        # Write the record to the CSV file immediately after receiving the response
            writer.writerow([start_name, end_name, dur])
        except requests.exceptions.RequestException as e:
            print(f"Error making API request: {e}")


print("CSV file 'output.csv' has been created and populated with data.")

Central Park
Sunset Park (Central)
Prospect Park
Upper East Side-Lenox Hill-Roosevelt Island
Gravesend (West)
Chelsea-Hudson Yards
Fort Wadsworth
East Elmhurst
Midtown South-Flatiron-Union Square
Sunnyside
Grasmere-Arrochar-South Beach-Dongan Hills
East Harlem (South)
University Heights (South)-Morris Heights
Midtown-Times Square
Carroll Gardens-Cobble Hill-Gowanus-Red Hook
West New Brighton-Silver Lake-Grymes Hill
Upper East Side-Yorkville
Cypress Hills
New Springville-Willowbrook-Bulls Head-Travis
Astoria (Central)
Bedford Park
North Corona
Sunset Park (East)-Borough Park (West)
Astoria (North)-Ditmars-Steinway
Coney Island-Sea Gate
Bushwick (East)
Arden Heights-Rossville
Annadale-Huguenot-Prince's Bay-Woodrow
Sunset Park (West)
East Midtown-Turtle Bay
Washington Heights (North)
Laurelton
Ozone Park
Mariner's Harbor-Arlington-Graniteville
Kensington
Murray Hill-Kips Bay
Windsor Terrace-South Slope
Bedford-Stuyvesant (East)
Westerleigh-Castleton Corners
Rosebank-Shore Acres-Park Hill
